In [303]:
import numpy as np
import random

In [304]:
# cross product of two vector fields
def cross(A, B):
    assert isinstance(A, np.ndarray) and isinstance(B, np.ndarray)
    _, N1, N2, N3 = A.shape
    assert A.shape == B.shape == (3, N1, N2, N3)

    return np.array([
        A[1] * B[2] - A[2] * B[1],
        A[2] * B[0] - A[0] * B[2],
        A[0] * B[1] - A[1] * B[0]
    ])

# shift a scalar field by one grid cell in the positive i direction
def shiftp(i, f):
    return np.roll(f, -1, axis=i)

# shift a scalar field by one grid cell in the negative i direction
def shiftm(i, f):
    return np.roll(f, 1, axis=i)

# half-shift a scalar field by one grid cell in the positive i direction
def half_shiftp(i, f):
    return 0.5 * (np.roll(f, -1, axis=i) + f)

# centered cross product of two vector fields
def ccp(A, B):
    assert isinstance(A, np.ndarray) and isinstance(B, np.ndarray)
    _, N1, N2, N3 = A.shape
    assert A.shape == B.shape == (3, N1, N2, N3)

    return np.array([
        A[1] * half_shiftp(0, B[2]) - A[2] * half_shiftp(0, B[1]),
        A[2] * half_shiftp(1, B[0]) - A[0] * half_shiftp(1, B[2]),
        A[0] * half_shiftp(2, B[1]) - A[1] * half_shiftp(2, B[0])
    ])

In [305]:
# forward difference along direction i = 0, 1, 2 of a scalar field f
def pdp(dxyz, i, f):
    (dx, dy, dz) = dxyz
    assert dx > 0 and dy > 0 and dz > 0
    assert isinstance(f, np.ndarray) and len(f.shape) == 3
    
    return (np.roll(f, -1, axis=i) - f) / dxyz[i]

# backward difference along direction i = 0, 1, 2 of a scalar field f
def pdm(dxyz, i, f):
    (dx, dy, dz) = dxyz
    assert dx > 0 and dy > 0 and dz > 0
    assert isinstance(f, np.ndarray) and len(f.shape) == 3
    
    return (f - np.roll(f, 1, axis=i)) / dxyz[i]

# gradient with forward differences
def gradp(dxyz, f):
    assert isinstance(f, np.ndarray) and len(f.shape) == 3
    
    return np.array([
        pdp(dxyz, 0, f),
        pdp(dxyz, 1, f),
        pdp(dxyz, 2, f)
    ])

# gradient with backward differences
def gradm(dxyz, f):
    assert isinstance(f, np.ndarray) and len(f.shape) == 3
    
    return np.array([
        pdm(dxyz, 0, f),
        pdm(dxyz, 1, f),
        pdm(dxyz, 2, f)
    ])

# divergence with forward differences
def divp(dxyz, v):
    assert isinstance(v, np.ndarray) and v.shape[0] == 3
    return pdp(dxyz, 0, v[0]) + pdp(dxyz, 1, v[1]) + pdp(dxyz, 2, v[2])

# divergence with backward differences
def divm(dxyz, v):
    assert isinstance(v, np.ndarray) and v.shape[0] == 3
    return pdm(dxyz, 0, v[0]) + pdm(dxyz, 1, v[1]) + pdm(dxyz, 2, v[2])

# curl with forward differences
def curlp(dxyz, v):
    assert isinstance(v, np.ndarray) and v.shape[0] == 3
    return np.array([
        pdp(dxyz, 1, v[2]) - pdp(dxyz, 2, v[1]),
        pdp(dxyz, 2, v[0]) - pdp(dxyz, 0, v[2]),
        pdp(dxyz, 0, v[1]) - pdp(dxyz, 1, v[0])
    ])

# curl with backward differences
def curlm(dxyz, v):
    assert isinstance(v, np.ndarray) and v.shape[0] == 3
    return np.array([
        pdm(dxyz, 1, v[2]) - pdm(dxyz, 2, v[1]),
        pdm(dxyz, 2, v[0]) - pdm(dxyz, 0, v[2]),
        pdm(dxyz, 0, v[1]) - pdm(dxyz, 1, v[0])
    ])

In [306]:
# solve the Poisson equation
def poisson_solve(dxyz, source):
    assert isinstance(source, np.ndarray) and len(source.shape) == 3

    # compute the Fourier coefficients corresponding to the Laplacian operator
    N1, N2, N3 = source.shape
    (dx, dy, dz) = dxyz
    arr1 = -4 / (dx ** 2) * (np.sin(np.pi / N1 * np.array(range(0, N1))) ** 2)
    arr2 = -4 / (dy ** 2) * (np.sin(np.pi / N2 * np.array(range(0, N2))) ** 2)
    arr3 = -4 / (dz ** 2) * (np.sin(np.pi / N3 * np.array(range(0, N3))) ** 2)
    lapl_dft = (arr1[:, np.newaxis, np.newaxis] +
                arr2[np.newaxis, :, np.newaxis] +
                arr3[np.newaxis, np.newaxis, :])
    lapl_dft[0, 0, 0] = 1.  # set (0, 0, 0) component to 1. because normally it is 0.

    rho_tilde = np.fft.fftn(source)  # compute the Fourier transform of charge density
    f_tilde = np.divide(rho_tilde, lapl_dft)
    f_tilde[0, 0, 0] = 0.  # set the constant term to zero
    return np.real(np.fft.ifftn(f_tilde))

In [307]:
# update the electric and magnetic fields
def update_EB(dt, dxyz, J, E, B):
    (dx, dy, dz) = dxyz
    assert dt > 0 and dx > 0 and dy > 0 and dz > 0
    assert isinstance(J, np.ndarray) and isinstance(E, np.ndarray) and isinstance(B, np.ndarray)
    E += dt * (curlp(dxyz, B) - J)
    B += dt * (-curlm(dxyz, E))

In [308]:
# compute the electromagnetic stress tensor
def stress_tensor(E, B):
    assert isinstance(E, np.ndarray) and isinstance(B, np.ndarray)
    _, N1, N2, N3 = E.shape
    assert E.shape == B.shape == (3, N1, N2, N3)
    
    sigma = np.zeros((3, 3, N1, N2, N3))

    for i in range(3):
        for j in range(3):
            sigma[i, j] += E[i] * shiftm(i, half_shiftp(j, E[j]))
            sigma[i, j] += B[j] * shiftm(i, half_shiftp(j, B[i]))
            if i == j:
                for k in range(3):
                    sigma[i, j] += -0.5 * (E[k] * shiftm(i, E[k]) + B[k] * B[k])
    
    return sigma

In [309]:
# simulate Maxwell's equations on an N1 x N2 x N3 grid with physical dimensions xsize, ysize, zsize for nsteps with time step dt,
# where the current density is randomized at each time step
# and the charge density is initialized at random and then updated to maintain charge conservation.
# output: rho_hist, J_hist, E_hist, B_hist storing the values of these fields at time steps 0, ..., nsteps.
def random_current_sim(N1, N2, N3, xsize, ysize, zsize, dt, nsteps):
    rho_hist = []
    J_hist = []
    E_hist = []
    B_hist = []
    
    (dx, dy, dz) = dxyz = (xsize / N1, ysize / N2, zsize / N3)  # compute grid spacings
    assert 1/dt**2 > 1/dx**2 + 1/dy**2 + 1/dz**2  # check that the CFL condition is satisfied

    # choose some random charge density, making sure the total charge is zero
    rho = np.random.rand(N1, N2, N3)
    rho -= np.sum(rho) / (N1 * N2 * N3)

    # initialize J at random
    J = np.random.rand(3, N1, N2, N3)

    # make sure the forward divergence of E is equal to rho
    E = gradm(dxyz, poisson_solve(dxyz, rho))

    # initialize B at random such that the backward divergence of B is zero
    B = curlm(dxyz, 10 * np.random.rand(3, N1, N2, N3))

    # record the current state of things
    rho_hist.append(rho.copy())
    J_hist.append(J.copy())
    E_hist.append(E.copy())
    B_hist.append(B.copy())

    # run the simulation for nsteps time steps
    for iteration in range(nsteps):
        # choose some current density J and update the charge density rho such that charge is conserved
        J = np.random.rand(3, N1, N2, N3)
        rho -= dt * divp(dxyz, J)
        
        # update the electric and magnetic fields
        update_EB(dt, dxyz, J, E, B)

        # record the current state of things
        rho_hist.append(rho.copy())
        J_hist.append(J.copy())
        E_hist.append(E.copy())
        B_hist.append(B.copy())

    return rho_hist, J_hist, E_hist, B_hist

In [310]:
# test that the time difference of the Poynting vector is minus the Lorentz force density plus a field gradient term
def poynting_test():
    # initialize simulation parameters
    (N1, N2, N3) = (10, 20, 7)
    (xsize, ysize, zsize) = (1., 3., 8.)
    dxyz = (xsize / N1, ysize / N2, zsize / N3)  # compute grid spacings
    dt = 0.03
    nsteps = 10

    rho_hist, J_hist, E_hist, B_hist = random_current_sim(N1, N2, N3, xsize, ysize, zsize, dt, nsteps)

    for n in range(1, nsteps):  # go through the time steps one by one
        # momentum density at time step n
        S = ccp(E_hist[n], B_hist[n-1])

        # momentum density at time step n + 1
        S_new = ccp(E_hist[n + 1], B_hist[n])

        # divergence of the stress tensor at time step n
        sigma = stress_tensor(E_hist[n], B_hist[n])
        div_sigma = np.array([divp(dxyz, sigma[:, i]) for i in range(3)])

        # centered electric field and Lorentz force density at time step n
        Ec = np.array([
            half_shiftp(0, E_hist[n][0]),
            half_shiftp(1, E_hist[n][1]),
            half_shiftp(2, E_hist[n][2])
        ])
        f = rho_hist[n] * Ec + ccp(J_hist[n + 1], B_hist[n])

        # compute to what degree is momentum not conserved
        difference = (S_new - S) / dt - div_sigma + f
        print(np.max(np.abs(difference)))

poynting_test()

3.033306938959868e-11
4.200018111077952e-11
3.077005317209114e-11
3.6862957131234e-11
5.567457606048265e-11
4.617817239704891e-11
5.8179239204037e-11
4.5639936274710635e-11
3.951150517877977e-11
